In [1]:
'''
    SEGUNDA AVALIAÇÃO - ALGEBRA LINEAR COMPUTACIONAL - 2020.2 - PROFESSOR ERITO MARQUES
    GRUPO: BRUNO/~MATRICULA~, ICARO/~MATRICULA~, LEANDRO/~MATRICULA~ E PEDRO/2018780326
    
    INSTRUÇÕES:
        1. AS RESPOSTAS DEVEM SER ENVIADAS NO FORMATO JUPYTER NOTEBOOK;
        2. OS TRABALHOS DEVEM SER FEITOS EM GRUPO. NÃO É PERMITIDO TROCA DE INFORMAÇÕES ENTRE OS GRUPOS;
        3. FIXADA A DATA DE ENTREGA, NÃO SERÃO ACEITOS TRABALHOS ENTREGUES FORA DESTA DATA.

    -----------------> FEITO <-----------------

    2) Implemente um algoritmo que leia uma matriz A (triangular) e um vetor b (de uma planilha excel) e resolva o sistema Ax = b por substituição para frente ou para trás conforme o caso.

    3) b) Resolva o sistema Ax = b usando a fatoração de Cholesky.

    4) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:
        a) Avalie se algum critério é satisfeito para aplicação dos métodos de Gauss-Seidel, Jacobi e SOR;
        b) Resolva o sistema Ax = b usando os métodos iterativos de Gauss-Seidel, Jacobi e SOR.

    5) e) Avalie se A é simétrica;
       g) Calcule a norma de linha, norma de Frobenius e norma coluna de A.

    6) a) Norma 1, norma 2 e norma infinito de x e y.

    7) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel). Encontre a fatoração LU de A.


    -----------------> A FAZER <-----------------

    1) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e avalie se:
        a) A é tridiagonal;
        b) A é triangular superior;
        c) A é triangular inferior;
        d) Traço de A é maior que 4.

    3) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e: 
        a) Avalie se é possível obter o fator de Cholesky;
        
    5) Implemente um algoritmo que leia uma matriz A (de uma planilha excel) e: 
        a) Calcule a transposta de A;
        b) Calcule o determinante de A;
        c) Calcule a matriz adjunta de A;
        d) Calcule o traço de A;
        f) Avalie se A é ortogonal;
        h) Avalie se A é positiva definida.

    6) Construa um algoritmo que receba dois vetores x e y e uma matriz A positiva definida (de uma planilha excel). Calcule: 
        
        b) Ângulo entre x e y;
        c) Norma de x e y induzida por A;
        d) produto interno de x e y.
    
    8) Construa um algoritmo que receba uma matriz A positiva (de uma planilha excel). Calcule:
        a) O número condição;
        b) Retorne uma mensagem ao usuário sobre sua interpretação.
    
    9) Construa um algoritmo que receba três matrizes: U, sigma e V (de uma planilha excel) e: 
        a) Avalie se o produto das três matrizes representa uma decomposição SVD de alguma matriz X;
        b) Caso a resposta a pergunta anterior seja negativa, retorne ao usuário uma explicação justificando esse fato.
    
    10) Construa um algoritmo que leia uma planilha em Excel com 4 colunas e calcule a estatística de Benford (para 1 dígito) para cada uma das colunas.

'''

import numpy as np
import pandas as pd
import normas
from cholesky import decomposicao_cholesky
from gauss import eliminacao_gauss
from fatoracaoLU import fatoracaoLU

''' BIBLIOTECA QUE NÃO ESTOU USANDO MAS PODE SER ÚTIL PARA EXIBIR AS MATRIZES/VETORES = display(Matrix(*variavel que representa o dado*) '''
from sympy import Matrix, init_printing
init_printing()

''' LEITURA DOS ARQUIVOS DA MATRIZ A (matriz_A.xlsx) E DO VETOR B (vetor_b.xlsx)'''
df_matriz = pd.read_excel(r"dados/matriz_A.xlsx", header=None) # Le o arquivo do excel que contém a matriz A sem cabeçalho
df_vetor = pd.read_excel(r"dados/vetor_b.xlsx", header=None) # Le o arquivo do excel que contém o vetor b sem cabeçalho

''' CONVERSÃO DOS DADOS EM ARRAYS DA NUMPY COM TYPE 'float64' '''
matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
vetor = np.ravel(np.array(df_vetor, dtype='f8')) # Converte o vetor b em um array da numpy

''' TESTE DO ALGORITMO DE DECOMPOSIÇÃO DE CHOLESKY E NORMA RESIDUAL (cholesky.py, normas.py) '''
delta_x = decomposicao_cholesky(matriz, vetor)
print("Vetor x resultado da Decomposição de Cholesky:\n",delta_x,"\n")
normas.residual(matriz, vetor, delta_x)
print("\n")

''' TESTE DO ALGORITMO DE ELIMINAÇÃO DE GAUSS E NORMA RESIDUAL (gauss.py, normas.py) '''
delta_x = eliminacao_gauss(matriz, vetor)
print("Vetor x resultado da Eliminação de Gauss:\n",delta_x,"\n")
normas.residual(matriz, vetor, delta_x)
print("\n")

''' TESTE DO ALGORITMO DE FATORAÇÃO LU E NORMA RESIDUAL (fatoracaoLU.py, normas.py)'''
delta_x = fatoracaoLU(matriz,vetor)
print("Vetor x resultado da Fatoração LU:\n",delta_x,"\n")
normas.residual(matriz, vetor, delta_x)

FileNotFoundError: [Errno 2] No such file or directory: '/dados/matriz_A.xlsx'